In [1]:
import requests,re
import sys
sys.path.append('/usr/local/lib/python2.7/dist-packages/')
import textblob
import langid
import logging
import collections
import pymongo,time
from secrets import *
from utils import *

### Summary
Script to scrape content from Facebook API.
Collects
1. Pages matching a keyword query
2. Recent posts from these pages (limit set by API)
3. Comments on these posts (it seems that likes on comments are not available)
4. Likes on these posts  

### Errors

Needs robust error handling due to errors with API requests. More detail [here](https://developers.facebook.com/docs/graph-api/using-graph-api/v2.4#errors)

* ``Error 500 {"error":{"message":"An unexpected error has occurred. Please retry your request later.","type":"OAuthException","is_transient":true,"code":2}}``
* ``Error 500 {"error":{"code":1,"message":"An unknown error occurred"}}``
* ``ConnectionError: ('Connection aborted.', gaierror(-2, 'Name or service not known'))``

### TODO

* ~~Get likes on comments~~
* ~~Have a restart page ID, so we can pick up if scraping fails~~
* Write a script to check for new posts on pages that already exist in DB 
* Add a record of when data was pulled  

Mongo tutorial [here](http://api.mongodb.org/python/current/tutorial.html)

### Set up query to grab pages

In [2]:

ands=[]
#ands.append('أوروبا') # Europe
#ands.append('ألمانيا')
#ands.append('المانيا') # Germany (two alternate spellings)

ors=[]
ors.append('مهاجرون') # Refugees
ors.append('مهاجرين') # Refugees
ors.append('المهاجرين') # The refugees
ors.append('المهاجرون') # The refugees
ors.append('المهاجرون') # The refugees
ors.append('مهاجرون') # refugees
ors.append('هجرة') # Migration
ors.append('الهجرة') # The migration
ors.append('أوروبا') # Europe
ors.append('سوريا') # Syria

#ors=[]
ors.append('مهرب') # Trafficker
ors.append('المتاجرين') # Traffickers

#QUERY='+'.join(ands)+'+'
QUERY='|'.join(ors)
#QUERY='syria'
#QUERY='سوريا'
#QUERY='أوروبا' # Europe
#QUERY=u'اوروبا'
#QUERY=u'اللجوء' # asylum
#QUERY=u'ملجأ' # asylum
# 'Europe' + 'ANY ['migration']...'
LIMIT=1000
# Page limit
postsLimit=250
# Hard limit from API=250

nSkip=5
# Hit API for query nSkip times
# before skipping

nWait=60
# Wait between API errors

postSleepTime=0.5
pageSleepTime=10
# Pause so API not thrashed

In [3]:
print QUERY

مهاجرون|مهاجرين|المهاجرين|المهاجرون|المهاجرون|مهاجرون|هجرة|الهجرة|أوروبا|سوريا|مهرب|المتاجرين


In [4]:
countCollections()

1151 pages
102108 posts
208796 comments
131039 likes


### Some boilerplate DB functions

In [5]:
def clearCollection(collection=None):
    
    all=False
    answer=True
    
    if not collection or collection.lower().strip()=='all':
        answer=raw_input('Clear all?')
        if answer.lower().strip() in ['y','yes']:
            all=True
            
    if all or collection=='likes':
        if not all:answer=raw_input('Clear likes?')
        if all or answer.lower().strip() in ['y','yes']:
            res=likesCollection.remove()
            print 'Cleared %d likes' % res['n']
            
    if all or collection=='pages':
        if not all:answer=raw_input('Clear pages?')
        if all or answer.lower().strip() in ['y','yes']:
            res=pagesCollection.remove()
            print 'Cleared %d pages' % res['n']

            
    if all or collection=='comments':
        if not all:answer=raw_input('Clear comments?')
        if all or answer.lower().strip() in ['y','yes']:
            res=commentsCollection.remove()
            print 'Cleared %d comments' % res['n']

            
    if all or collection=='posts':
        if not all:answer=raw_input('Clear posts?')
        if all or answer.lower().strip() in ['y','yes']:
            res=postsCollection.remove()
            print 'Cleared %d posts' % res['n']


In [6]:
countCollections()

1151 pages
102108 posts
208796 comments
131039 likes


In [7]:
def isCommentInDb(id):
    '''
    Tests if a comment is in comments collection
    Returns Bool
    '''
    nMatches=commentsCollection.find({'id':id}).count()
    if nMatches==0:
        return False
    elif nMatches==1:
        return True
    else:
#        logging.warning('Duplicate comment %s' % id)
        return True

In [8]:
def addTimestampToPage(id):
    '''
    Adds current timestamp to page 
    '''
    pagesCollection.update({'id':id},{'$set':{'checked':[int(time.time())]}})

In [9]:
def updatePageTimestamp(id):
    '''
    Updates timestamp of page 
    '''
    pagesCollection.update({'id':id},{'$addToSet':{'checked':int(time.time())}})

In [27]:
def isPostInDb(id):
    '''
    Tests if a post is in post collection
    Returns Bool
    '''
    nMatches=postsCollection.find({'id':id}).count()
    if nMatches==0:
        return False
    elif nMatches==1:
        return True
    else:
#        logging.warning('Duplicate post %s' % id)
        return True

In [28]:
def isPageInDb(id):
    '''
    Tests if a page is in pages collection
    Returns Bool
    '''
    nMatches=pagesCollection.find({'id':id}).count()
    
    if nMatches==0:
        return False
    elif nMatches==1:
        return True
    else:
#        logging.warning('Duplicate page %s' % id)
        return True

### How to deal with API errors

In [10]:
def handleResult(statusCode,returnText):
    '''
    Parses API call result to determine if successful
    or to wait or abandon
    Returns success,skip (both Bool)
    '''
    if statusCode==200:
        # OK
        return True,False
    
    if statusCode in [102,10,463,467]:
        # Access token expired
        logging.warning('API error: %d %s' % (statusCode,returnText))
        return False,True
    elif statusCode in [2,4,17,341,500]:
        # Wait and retry
        logging.warning('API error - waiting: %d %s' % (statusCode,returnText))
        return False,False
    elif statusCode in [506,1609005]:
        # Skip
        logging.warning('API error - skipping: %d %s' % (statusCode,returnText))
        return False,True
    else:
        logging.warning('API error - unknown code %d %s' % (statusCode,returnText))
        return False, True

### Start by looping through pages

In [11]:
######################################################
success=None
nAttempts=0

temp='https://graph.facebook.com/search?q=%s&limit=%d&type=page&access_token=%s' % (QUERY, LIMIT, ACCESSTOKEN)

while not success:
    # Keep looping if unsuccessful
    r=requests.get(temp)
    success,skip=handleResult(r.status_code,r.text)
    # Try, find out if successful or should skip
    
    if skip or nAttempts==nSkip:
        # If tried nSkip times or if should skip
        r={'data':[],'paging':None}
        if nAttempts==nSkip:
            logging.warning('Skipping after %d attempts' % nAttempts)
            break
    time.sleep(nWait)
    nAttempts+=1
######################################################

In [ ]:
getPostsFromPage(307091372652912)

In [12]:
def getPages(data,paging,restart=None):
    '''
    Takes data returned from an API call searching for pages
    '''
    
    if paging:
        if paging.get('next'):
            logging.warning('Paging needed')
    
    for n,d in enumerate(data):
        
        if (restart and d['id']==str(restart)) or (not restart):
            # If restart id defined then wait until we find it
            # if not add straight in
            restart=None
        
            time.sleep(pageSleepTime)

            print n,d['name']

            if not langid.classify(d['name'])[0]=='en':
                try:
                    enName=textblob.TextBlob(d['name']).translate().string
                    enName=clean(enName)
                    print 'Translates: ',enName
                except:
                    logging.warning('Translation failed')
                    enName=None
            else:
                enName=None

            print 'http://fb.com/'+d['id'],d.get('category')
            res=getPageInfo(d['id'],raw=True)

            res['name']=clean(d['name'])
            res['about']=clean(d.get('about'))
            res['description']=clean(d.get('description'))

            ######################################################################
            posts,comments,likes=getPostsFromPage(d['id'],limit=postsLimit,raw=False)
            # Get posts,comments,likes from that page
            nAdded=nAlready=0
            for post in posts:
                if not isPostInDb(post['id']):
                    addPostToDb(post)
                    nAdded+=1
                else:
                    #logging.warning('Post %s already in DB' % post['id'])
                    nAlready+=1
            logging.warning('%d posts added (%d already in DB)' % (nAdded,nAlready))
    #        addCommentsToDb(comments)

    #        addLikesToDb(likes)
            ######################################################################

            if enName:
                res['name_en']=enName

            category=d.get('category')
            if category:
                res['category']=category

            print res.keys()
            if not isPageInDb(d['id']):
                addPageToDb(res)
            else:
                logging.info('Page %s already in DB' % d['id'])
            addTimestampToPage(d['id'])

        else:
            logging.warning('Skipping page %s. Waiting for %s to restart' % (d['id'],restart))

In [20]:
def getPostsFromPage(pageId,raw=False,limit=100):
    '''
    Requests list of posts, list of comments and 
    list of likes from a page
    Returns a list of JSON objects
    or if raw=True, a string description of posts
    '''
    
    logging.info('Getting posts,comments,likes for page %s' % pageId)
    
    tempUrl='https://graph.facebook.com/%s/posts?&limit=%d&access_token=%s' % (pageId,postsLimit,ACCESSTOKEN)
    
    out=[]
    outFull=[]
    
    comments=None
    likes=None
    
    r=requests.get(tempUrl)
    ######################################################
    success=None
    nAttempts=0

    while not success:
        # Keep looping if unsuccessful
        r=requests.get(tempUrl)
        success,skip=handleResult(r.status_code,r.text)
        # Try, find out if successful or should skip

        if skip or nAttempts==nSkip:
            # If tried nSkip times or if should skip
            r={'data':[],'paging':None}
            if nAttempts==nSkip:
                logging.warning('Skipping posts after %d attempts' % nAttempts)
                return ([],[],[])
        time.sleep(nWait)
        nAttempts+=1
    ######################################################
    
    for n,d in enumerate(r.json()['data']):
        
        time.sleep(postSleepTime)

        
        name=d.get('name')
        id=d.get('id')
        print 'Post %d %s %s' % (n,name,id)
        
        message=d.get('message')
        if message:
            message=clean(message)
        else:
            logging.warning('No message for post %s' % d['id'])
        
        description=d.get('description')
        if description:
            description=clean(description)
        else:
            logging.warning('No description for post %s' % d['id'])
        
        caption=d.get('caption')
        if caption:
            caption=clean(caption)
        else:
            logging.warning('No caption for post %s' % d['id'])
        
        d['page_id']=pageId
        d['retrieved']=time.time()
        
        if d.get('icon'):del d['icon']
        if d.get('picture'):del d['picture']
        if d.get('privacy'):del d['privacy']
        # Don't need these
        
        try:
            shareCount=d['shares']['count']
            d['shares']=shareCount
        except:
            pass
        # Simplify this
        
        if message:
#            print message
            out.append(message)
            if not langid.classify(message)[0]=='en':
                try:
                    enMessage=textblob.TextBlob(message).translate().string
                    enMessage=clean(enMessage)
                    out.append('==>'+enMessage+'---------')
                    d['en_message']=enMessage
                except:
                    logging.warning('Translation failed')
                    enMessage=None
        if description:
            out.append(description)
            if not langid.classify(description)[0]=='en':
                try:
                    enDescription=textblob.TextBlob(description).translate().string
                    enDescription=clean(enDescription)
                    out.append('==>'+enDescription+'---------')
                    d['en_description']=enDescription
                except:
                    logging.warning('Translation failed')
                    enDescription=None
        if caption:
            out.append(caption)
            if not langid.classify(caption)[0]=='en':
                try:
                    enCaption=textblob.TextBlob(caption).translate().string
                    enCaption=clean(enCaption)
                    out.append('==>'+enCaption+'---------')
                    d['en_caption']=enCaption
                except:
                    enCaption=None
                    
        try:
            comments=d['comments']
            del d['comments']
        except:
            comments=None
        
        if comments:
            logging.info('Getting comments...')
            commentData=getComments(comments,pageId)
            # This does all the paging
#            for c in commentData['data']:
#                print 'Comments data:',c.keys()
#                print 'Likes:',c[u'user_likes'],c.get('likes')
            addCommentsToDb(commentData)
            
            # TODO get comment likes
            
        try:
            likes=d['likes']
            del d['likes']
        except:
            likes=None
        
        if likes:
            logging.info('Getting likes...')
            likeData=getLikes(likes,pageId,id)
            # This does all the paging
            addLikesToDb(likeData)

        
        outFull.append(d)       
    
    if raw:
        '\n'.join(out)
        pass # return string
    else:
        return outFull,comments,likes

In [21]:
def getLikes(likes,pageId,id):
    '''
    Takes a dictionary of like data from API with keys
    [paging,data], pageId and post id. If paging information is present, keep 
    requesting pages
    '''
    
    
    if likes.get('paging'):
        current=likes
        
        while current['paging'].get('next'):
            logging.info('Paging likes... %s' % current['paging']['next'])
            current=getNextLikes(current['paging']['next'])
            if current:
                print 'Got next page of likes (current) (%d so far)' % len(likes['data'])
                likes['data'].extend(current['data'])
            else:
                break
            # TODO better error handling
                
        print 'Got %d likes after paging' % len(likes['data'])
#    print 'Likes type %s' % type(likes)
#    print likes.keys()
    
    for like in likes['data']:
        like['id']='%s_%s' % (id,like['id'])
        # Make a unique like ID made up of post id_likeid
        like['parent_id']=id
        # Keep parent id of comment/post for getting most liked content
    
#    print 'Likes',likes.keys()
#    print likes.get('paging')
#    print likes['data'][0]
    return likes

In [22]:
def getComments(comments,pageId):
    '''
    Takes a dictionary of comment data from API with keys
    [paging,data]. If paging information is present, keep
    requesting pages
    '''
    
    if comments.get('paging'):
        
        current=comments
        
        while current['paging'].get('next'):
            logging.info('Paging comments...')
            current=getNextComments(current['paging']['next'])
            comments['data'].extend(current['data'])
            
            if not current.get('paging'):
                break
    return comments

In [23]:
def getNextComments(nextToken):
    '''
    Given a paging token for next page of likes returns the raw data
    '''
    res=requests.get(nextToken)
    
    if not res.status_code==200:
        logging.warning('Error with next comments data %d' % res.status_code)
        return None
    else:
        return res.json()

In [24]:
def getNextLikes(nextToken):
    '''
    Given a paging token for next page of likes returns the raw data
    '''
#    logging.warning('Getting next likes: %s' % nextToken)
    res=requests.get(nextToken)
    
    if not res.status_code==200:
        logging.warning('Error with next likes data %d %s' % (res.status_code,res.text))
        return None
    else:
        return res.json()

In [25]:
def getPageInfo(pageId,raw=False):
    '''
    Requests info for a page by ID
    Returns the info either as a JSON object
    or if raw=True as a string to be printed
    '''
    tempUrl='https://graph.facebook.com/v2.4/'+pageId+'?fields=about,description,location,phone,talking_about_count,\
    engagement,start_info,likes,website&access_token='+ACCESSTOKEN

    res=requests.get(tempUrl)
    
    
    if not res.status_code==200:
        logging.warning('Request failed: %d %s' % (res.status_code,res.text))
    
    res=res.json()
    res['retrieved']=time.time()
    
    description=res.get('description')
    if description:
        description=clean(description)
    else:
        logging.warning('No description for page %s' % pageId)
        logging.warning('Keys %s' % res.keys())
    
    engagement=res.get('engagement')
    if engagement:
        if engagement.get(u'count'):
            res['engagement']=str(res['engagement']['count'])
    
    start_info=res.get('start_info')
    if start_info:
        date=start_info.get(u'date')
        if date:
            res['start_info_clean']=str(res['start_info']['date']['year'])
            if res['start_info']['date'].get('month'):
                res['start_info_clean']+='/'+str(res['start_info']['date']['month'])
                if res['start_info']['date'].get('day'):
                    res['start_info_clean']+='/'+str(res['start_info']['date']['day'])

        else:
            del res['start_info']
    
    for k,v in res.items():
        if type(v) in [unicode,str]:

            if not langid.classify(v)[0]=='en':

                try:
                    res[k+'_en']=textblob.TextBlob(v).translate().string
                except:
                    logging.warning('Translation failed')
                # Create a new dictionary entry with the translation
    if raw:    
        return res
    else:
#        print res.json().items()
        return '\n'.join([k+'\t\t'+unicode(v) for k,v in res.items()])+'\n================'

In [ ]:
getPages(r.json()['data'],r.json().get('paging'),restart=None)